In [147]:
from dotenv import load_dotenv

load_dotenv()

True

In [166]:
from langchain_teddynote import logging

logging.langsmith("Recommend")

LangSmith 추적을 시작합니다.
[프로젝트명]
Recommend


### 모듈 import

In [149]:
import pandas as pd
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda 
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document
from datetime import datetime
from langchain_community.document_loaders import JSONLoader
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_anthropic import ChatAnthropic
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema import BaseOutputParser
import json

## 1. 대분류 :의도파악하기

(가격)  
- Upstage Solar pro :  0.25(preview동안 0원)  
- gpt-4o-mini-2024-07-18 : input 0.15, output 0.6  
- claude-3-5-sonnet-20240620 : input 3, output 15  
- GPT-4o 2024-08-06 : input 2.5, output 10

In [48]:
# Upstage : Solar-1-mini

llm = ChatUpstage()
# 주문일때 키워드 뽑는건 잘함. 
# 1초 미만(0~0.7초)
# {'분류': '기타', '키워드': []}, {'분류': '추천', '키워드': []} 등 주문 아닌데도 키워드 항목 만들고 빈 리스트,,,
# "처음부터 주문 다시할게" -> 취소(O) but 주문으로 판단

In [82]:
# chatGPT 4o mini

llm = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18", temperature=0.3)  #temperature 바꿔서도 실행해보기
# 1초대 속도, 근데 딱 하나가 최대시간(3초)차지
# claude와 정확도면에서 크게 차이나진 않음 -> 우선선택!


In [70]:
# Claude 3.5 sonnet
llm = ChatAnthropic(model_name="claude-3-5-sonnet-20240620")
# 2초씩 걸린다... 정확도는 높음

In [83]:
intent_chain = PromptTemplate.from_template("""
아래 질문을 보고 사용자가 원하는 의도를 '주문', '추천', '취소', '결제', '기타' 중 하나로 정확하게 분류하세요.

분류 기준:
- 주문: 특정 메뉴를 정확히 주문하려는 경우 (예: '빅맥 하나 주세요', '감자튀김 추가')
- 추천: 특정 메뉴 이름 대신 추천을 요청하거나 메뉴 선택에 도움을 원하는 경우 (예: '뭐가 맛있나요?', '매콤한 음식 추천해줘')
- 취소: 이전에 진행된 주문을 취소하려는 경우 (예: '주문 취소해 주세요', '아까 주문한 것 취소하고 싶어요')
- 결제: 주문 완료 후 결제를 요청하는 경우 (예: '결제할게요', '주문 끝났어')
- 기타: 위의 네 가지 분류에 해당하지 않는 경우 (예: '화장실은 어디인가요?', '영업 시간은 언제인가요?')

<질문>
{question}
</질문>

질문의 분류와 해당 질문에 포함된 주요 키워드를 JSON 형식으로 출력하세요.

조건:
- 질문의 분류와 해당 질문에 포함된 주요 키워드를 딕셔너리 형식으로 출력하세요.
- 분류가 '주문'일 때만 키워드를 추출하세요. 다른 분류에서는 키워드를 출력하지 마세요
- 주문이 세트메뉴인 경우, 버거명에 세트메뉴 키워드를 포함하세요

예시 출력: {{"분류": "주문", "키워드": "빅맥"}}
예시 출력: {{"분류": "추천"}}
""") | llm | StrOutputParser()

In [51]:
# 추가적인 파싱 작업: 문자열을 딕셔너리로 변환
import json

def parse_response(response_str):
    try:
        response_dict = json.loads(response_str)
    except json.JSONDecodeError:
        response_dict = {"error": "파싱 오류", "원본": response_str}
    return response_dict

의도한게 '주문'

In [52]:
intent = intent_chain.invoke("불고기 버거 하나랑 후렌치후라이 하나줘")
print(parse_response(intent))
# 주문

{'분류': '주문', '키워드': ['불고기 버거', '후렌치후라이']}


In [53]:
intent = intent_chain.invoke("불고기 버거 세트에 후렌치후라이 하나랑 콜라주고, 상하이 치킨 스낵랩도 하나 추가해줘")
print(parse_response(intent))
# 주문

{'분류': '주문', '키워드': ['불고기 버거 세트', '후렌치후라이', '콜라', '상하이 치킨 스낵랩']}


In [86]:
intent = intent_chain.invoke("세트로 불고기 버거랑 후렌치후라이 하나랑 콜라주고, 상하이 치킨 스낵랩도 하나 추가해줘")
print(parse_response(intent))
# 주문

{'분류': '주문', '키워드': '불고기 버거 세트, 후렌치후라이, 콜라, 상하이 치킨 스낵랩'}


In [54]:
intent = intent_chain.invoke("불고기 버거 하나만 줘")
print(parse_response(intent))
# 주문

{'분류': '주문', '키워드': '불고기 버거'}


In [55]:
intent = intent_chain.invoke("불고기 버거세트 하나만 줘")
print(parse_response(intent))
# 주문

{'분류': '주문', '키워드': '불고기 버거세트'}


In [56]:
intent = intent_chain.invoke("불고기 버거 세트로 하나만 줘")
print(parse_response(intent))
# 주문

{'분류': '주문', '키워드': '불고기 버거 세트'}


In [57]:
parse_intent = parse_response(intent)
print(type(parse_intent))

<class 'dict'>


의도한게 '추천'

In [58]:
intent = intent_chain.invoke("주문할게")
print(parse_response(intent))
# 추천, 주문

{'분류': '주문'}


In [59]:
intent = intent_chain.invoke("치킨버거 하나만 줘")
print(parse_response(intent))
# 추천, 주문

{'분류': '주문', '키워드': '치킨버거'}


In [60]:
intent = intent_chain.invoke("치킨버거 아닌것도 있나?")
print(parse_response(intent))
# 추천

{'분류': '추천'}


In [61]:
intent = intent_chain.invoke("뭐 먹을지 잘 모르겠어")
print(parse_response(intent))
# 추천

{'분류': '추천'}


In [62]:
intent = intent_chain.invoke("매콤한거 먹을래")
print(parse_response(intent))
# 추천

{'분류': '추천'}


의도한게 '취소'

In [63]:
intent = intent_chain.invoke("방금 얘기한거 주문 안할게")
print(parse_response(intent))
# 취소

{'분류': '취소'}


In [64]:
intent = intent_chain.invoke("처음부터 주문 다시할게")
print(parse_response(intent))
# 취소

{'분류': '취소'}


의도한게 '결제'

In [65]:
intent = intent_chain.invoke("주문 끝났어")
print(parse_response(intent))
# 결제

{'분류': '결제'}


In [66]:
intent = intent_chain.invoke("주문완료")
print(parse_response(intent))
# 결제

{'분류': '결제'}


In [67]:
intent = intent_chain.invoke("계산해줘")
print(parse_response(intent))
# 결제

{'분류': '결제'}


'기타'를 의도

In [68]:
intent = intent_chain.invoke("이거 어떻게 하는거야?")
print(parse_response(intent))
# 기타

{'분류': '기타'}


In [69]:
intent = intent_chain.invoke("화장실은 어디야?")
print(parse_response(intent))
# 기타

{'분류': '기타'}


## task별 구분

주문(빠른주문) : 명확한 이름을 아는 경우 JSON 형식으로 출력. 명확한 이름을 모르는 경우 LLM으로 한번 걸러서 주문 진행.  
결제, 취소는 코딩으로 해결  
기타 : LLM 무한가동시켜서 의도분류 task중 하나 선택하도록 유도  

## 2. 빠른주문

### 전체코드

필요작업   
LLM에서 추측한 메뉴가 여러개일 때, 둘 중 맞는게 있는지 확인  
맨 뒤에 의도분류 연결  
GPT, claude 모델 비교


In [146]:
import json

llm = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18", temperature=0.3)
# llm = ChatAnthropic(model_name="claude-3-5-sonnet-20240620")

question = input("질문을 입력하세요: ")

# 의도추측 LLM
def parse_response(response_str):
    try:
        response_dict = json.loads(response_str)
    except json.JSONDecodeError:
        response_dict = {"error": "파싱 오류", "원본": response_str}
    return response_dict

intent_chain = PromptTemplate.from_template("""
아래 질문을 보고 사용자가 원하는 의도를 '주문', '추천', '취소', '결제', '기타' 중 하나로 정확하게 분류하세요.

분류 기준:
- 주문: 특정 메뉴를 정확히 주문하려는 경우 (예: '빅맥 하나 주세요', '감자튀김 추가')
- 추천: 특정 메뉴 이름 대신 추천을 요청하거나 메뉴 선택에 도움을 원하는 경우 (예: '무엇이 맛있나요?', '매콤한 음식 추천해줘')
- 취소: 이전에 진행된 주문을 취소하려는 경우 (예: '주문 취소해 주세요', '아까 주문한 것 취소하고 싶어요')
- 결제: 주문 완료 후 결제를 요청하는 경우 (예: '결제할거야', '주문 완료')
- 기타: 위의 네 가지 분류에 해당하지 않는 경우 (예: '화장실은 어디인가요?', '영업 시간은 언제인가요?')

<질문>
{question}
</질문>

질문의 분류와 해당 질문에 포함된 주요 키워드를 JSON 형식으로 출력하세요.

조건:
- 질문의 분류와 해당 질문에 포함된 주요 키워드를 딕셔너리 형식으로 출력하세요.
- 분류가 '주문'일 때만 키워드를 출력하세요. 다른 분류에서는 키워드를 출력하지 마세요
- 주문이 세트메뉴인 경우, 버거명에 세트메뉴 키워드를 포함하세요

예시 출력: {{"분류": "주문", "키워드": "빅맥"}}
예시 출력: {{"분류": "추천"}}
""") | llm | StrOutputParser()

# 이름추측 LLM
class CustomOutputParser(BaseOutputParser):
    def parse(self, text: str) -> dict:
        # 한심 'answer' 키로 '정답'을 반환
        return {"answer": text.strip()}

output_parser = CustomOutputParser()

name_chain = PromptTemplate.from_template("""
사용자의 질문과 메뉴 리스트를 비교하여 사용자의 질문과 비슷한 메뉴가 있는지 확인하세요.
비슷한 이름의 메뉴가 있다면 비슷한 메뉴를 모두 출력하세요
비슷한 메뉴가 없다면 "없음"이라고 출력하세요.
출력 시 메뉴 이름만 출력하세요

<메뉴 리스트>
['맥크리스피 스리라차 마요', '베토디 스리라차 마요', '맥스파이시 상하이 버거', '토마토 치즈 비프 버거', '더블 쿼터파운더 치즈', '쿼터파운더 치즈', '빅맥', '맥크리스피 디럭스 버거', '맥크리스피 클래식 버거', '1955 버거', '맥치킨 모짜렐라', '맥치킨', '더블 불고기 버거', '불고기 버거', '슈슈 버거', '슈비 버거', '베이컨 토마토 디럭스', '치즈버거', '더블 치즈버거', '트리플 치즈버거', '햄버거', '소시지 스낵랩', '맥윙', '맥윙콤보', '코울슬로', '상하이 치킨 스낵랩', '골든 모짜렐라 치즈스틱', '후렌치 후라이', '맥너겟', '맥스파이시치킨 텐더', '해쉬 브라운', '애플 파이', '베리 스트로베리 맥플러리', '오레오 맥플러리', '딸기 오레오 맥플러리', '초코 오레오 맥플러리', '스트로베리콘', '아이스크림콘', '바닐라 선데이 아이스크림', '초코 선데이 아이스크림', '딸기 선데이 아이스크림', '오레오 아포가토', '바닐라 라떼', '아이스 바닐라 라떼', '카페라떼', '디카페인 카페라떼', '아이스 카페라떼', '디카페인 아이스 카페라떼', '아메리카노', '디카페인 아메리카노', '아이스 아메리카노', '디카페인 아이스 아메리카노', '카푸치노', '디카페인 카푸치노', '드립 커피', '아이스 드립 커피', '코카-콜라', '코카-콜라 제로', '스프라이트', '환타', '바닐라 쉐이크 Medium', '딸기 쉐이크 Medium', '초코 쉐이크 Medium', '오렌지 주스', '생수']
</메뉴 리스트>

<질문>
{question}
</질문>

""") | llm | output_parser

# 빠른주문 모듈 전처리 함수들
def process_keyword(keyword):
    keyword = ''.join(keyword)
    keyword = keyword.replace("세트", "").strip()
    keyword = keyword.replace("버거", "").strip()    
    return keyword

def save_menu(keyword):
    menu_data = None
    for item in data:
        if item['page_content']['name'] == keyword:
            menu_data = {
                "name": item['page_content']['name'],
                "category": item['page_content']['category'],
                "description": item['page_content']['description'],
                "price": item['page_content']['price'],
                "nutrition": item['page_content']['nutrition'],
                "origin": item['page_content']['origin']
            }
            break
    if menu_data:
        menu_data_json = json.dumps(menu_data, ensure_ascii=False, indent=4)
        return menu_data_json
    else:
        return None

def find_menu(keyword, menu_list):
    for menu in menu_list:
        if keyword == menu.replace('버거', '').strip():
            return save_menu(menu)
    return None

def add_to_cart(order_data, num):
    cart = {}
    cart["name"] = order_data["name"]
    cart["num"] = num
    cart["price"] = order_data["price"]
    cart["set_menu"] = order_data["set_menu"]
    if order_data["set_menu"]:
        cart["set_price"] = order_data["price"]  #나중에 set_price로 변경예정
    shopping_cart.append(cart)

intent = intent_chain.invoke(question)
parsed_intent = parse_response(intent)
category = parsed_intent.get('분류')
intent_keyword = parsed_intent.get('키워드')

### 빠른주문 모듈
recommend = False
set_menu = False
menu_data = []
shopping_cart = []
unrecognized_keywords = []

if category == '주문':
    try:
        keyword_list = intent_keyword.split(',')
        for keyword in keyword_list:
            n_keyword = process_keyword(keyword)
            llm_keyword = process_keyword(name_chain.invoke(keyword)["answer"])
            menu_data_json = find_menu(n_keyword, menu_list) or find_menu(llm_keyword, menu_list)
            if menu_data_json is None:
                unrecognized_keywords.append(keyword)
                recommend = True
            else:
                menu_data_json = json.loads(menu_data_json) if isinstance(menu_data_json, str) else menu_data_json
                if "세트" in keyword:
                    menu_data_json['set_menu'] = True
                else:
                    pass
                menu_data.append(menu_data_json)
    except KeyError as e:
        print(f"키워드 없음: {e}")
        recommend = True
    except TypeError as e:
        print(f"TypeError 발생: {e}")
        recommend = True
    except Exception as e:  # 기타 에러 처리
        print(f"에러 발생: {e}")
        recommend = True
else:
    pass
    
if menu_data:
    for order_data in menu_data:
        num = input(f"{order_data['name']} 메뉴가 맞으신가요? 수량은 몇 개 드릴까요?")                
        add_to_cart(order_data, num)
if unrecognized_keywords:
    print(f"선택하신 메뉴 중 확인할 수 없는 항목이 있습니다: {unrecognized_keywords}. 추천 챗봇을 연결해 드리겠습니다.")

print(f"shopping_cart : {shopping_cart}")

선택하신 메뉴 중 확인할 수 없는 항목이 있습니다: [' 콜라']. 추천 챗봇을 연결해 드리겠습니다.
shopping_cart : [{'name': '불고기 버거', 'num': '1', 'price': '4200', 'set_menu': True, 'set_price': '4200'}, {'name': '후렌치 후라이', 'num': '1', 'price': '3000', 'set_menu': False}, {'name': '상하이 치킨 스낵랩', 'num': '1', 'price': '4000', 'set_menu': False}]


가격구성....헬게이트.....  
SHOPPING CART [[{}]] 최종 주문LIST에 또 APPEND하기  
'세트'이면 세트의 갯수를 물어보도록 하기  
세트, 단품 나눈데서 아예 메뉴 추가하면 되려나?  
버거가 세트메뉴이면 후렌치 후라이, 콜라 하나씩 엮어서 세트가격 묶고, 추가메뉴는 추가가격 계산하고, 세트구성이 완전히 되지 않으면 정정하는 작업을 '결제'모듈에서 실행

#### 디버깅과정

In [ ]:
import json

llm = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18", temperature=0.3)
# llm = ChatAnthropic(model_name="claude-3-5-sonnet-20240620")

question = input("질문을 입력하세요: ")

# # 메뉴명 추출
# file_path = '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/menu_1014.json'

# with open(file_path, 'r', encoding='utf-8') as file:
#     data = json.load(file)
# menu_list = []
# for item in data:
#     menu_name = item['page_content']['name']
#     menu_list.append(menu_name)

# 의도추측 LLM
def parse_response(response_str):
    try:
        response_dict = json.loads(response_str)
    except json.JSONDecodeError:
        response_dict = {"error": "파싱 오류", "원본": response_str}
    return response_dict

intent_chain = PromptTemplate.from_template("""
아래 질문을 보고 사용자가 원하는 의도를 '주문', '추천', '취소', '결제', '기타' 중 하나로 정확하게 분류하세요.

분류 기준:
- 주문: 특정 메뉴를 정확히 주문하려는 경우 (예: '빅맥 하나 주세요', '감자튀김 추가')
- 추천: 특정 메뉴 이름 대신 추천을 요청하거나 메뉴 선택에 도움을 원하는 경우 (예: '무엇이 맛있나요?', '매콤한 음식 추천해줘')
- 취소: 이전에 진행된 주문을 취소하려는 경우 (예: '주문 취소해 주세요', '아까 주문한 것 취소하고 싶어요')
- 결제: 주문 완료 후 결제를 요청하는 경우 (예: '결제할거야', '주문 완료')
- 기타: 위의 네 가지 분류에 해당하지 않는 경우 (예: '화장실은 어디인가요?', '영업 시간은 언제인가요?')

<질문>
{question}
</질문>

질문의 분류와 해당 질문에 포함된 주요 키워드를 JSON 형식으로 출력하세요.

조건:
- 질문의 분류와 해당 질문에 포함된 주요 키워드를 딕셔너리 형식으로 출력하세요.
- 분류가 '주문'일 때만 키워드를 출력하세요. 다른 분류에서는 키워드를 출력하지 마세요
- 주문이 세트메뉴인 경우, 버거명에 세트메뉴 키워드를 포함하세요

예시 출력: {{"분류": "주문", "키워드": "빅맥"}}
예시 출력: {{"분류": "추천"}}
""") | llm | StrOutputParser()

# 이름추측 LLM
class CustomOutputParser(BaseOutputParser):
    def parse(self, text: str) -> dict:
        # 한심 'answer' 키로 '정답'을 반환
        return {"answer": text.strip()}

output_parser = CustomOutputParser()

name_chain = PromptTemplate.from_template("""
사용자의 질문과 메뉴 리스트를 비교하여 사용자의 질문과 비슷한 메뉴가 있는지 확인하세요.
비슷한 이름의 메뉴가 있다면 비슷한 메뉴를 모두 출력하세요
비슷한 메뉴가 없다면 "없음"이라고 출력하세요.
출력 시 메뉴 이름만 출력하세요

<메뉴 리스트>
['맥크리스피 스리라차 마요', '베토디 스리라차 마요', '맥스파이시 상하이 버거', '토마토 치즈 비프 버거', '더블 쿼터파운더 치즈', '쿼터파운더 치즈', '빅맥', '맥크리스피 디럭스 버거', '맥크리스피 클래식 버거', '1955 버거', '맥치킨 모짜렐라', '맥치킨', '더블 불고기 버거', '불고기 버거', '슈슈 버거', '슈비 버거', '베이컨 토마토 디럭스', '치즈버거', '더블 치즈버거', '트리플 치즈버거', '햄버거', '소시지 스낵랩', '맥윙', '맥윙콤보', '코울슬로', '상하이 치킨 스낵랩', '골든 모짜렐라 치즈스틱', '후렌치 후라이', '맥너겟', '맥스파이시치킨 텐더', '해쉬 브라운', '애플 파이', '베리 스트로베리 맥플러리', '오레오 맥플러리', '딸기 오레오 맥플러리', '초코 오레오 맥플러리', '스트로베리콘', '아이스크림콘', '바닐라 선데이 아이스크림', '초코 선데이 아이스크림', '딸기 선데이 아이스크림', '오레오 아포가토', '바닐라 라떼', '아이스 바닐라 라떼', '카페라떼', '디카페인 카페라떼', '아이스 카페라떼', '디카페인 아이스 카페라떼', '아메리카노', '디카페인 아메리카노', '아이스 아메리카노', '디카페인 아이스 아메리카노', '카푸치노', '디카페인 카푸치노', '드립 커피', '아이스 드립 커피', '코카-콜라', '코카-콜라 제로', '스프라이트', '환타', '바닐라 쉐이크 Medium', '딸기 쉐이크 Medium', '초코 쉐이크 Medium', '오렌지 주스', '생수']
</메뉴 리스트>

<질문>
{question}
</질문>

""") | llm | output_parser

# 빠른주문 모듈 전처리 함수들
def process_keyword(keyword):
    keyword = ''.join(keyword)
    keyword = keyword.replace("세트", "").strip()
    keyword = keyword.replace("버거", "").strip()    
    return keyword

def save_menu(keyword):
    menu_data = None
    for item in data:
        if item['page_content']['name'] == keyword:
            menu_data = {
                "name": item['page_content']['name'],
                "category": item['page_content']['category'],
                "description": item['page_content']['description'],
                "price": item['page_content']['price'],
                "nutrition": item['page_content']['nutrition'],
                "origin": item['page_content']['origin']
            }
            break
    if menu_data:
        menu_data_json = json.dumps(menu_data, ensure_ascii=False, indent=4)
        return menu_data_json
    else:
        return None

def find_menu(keyword, menu_list):
    for menu in menu_list:
        if keyword == menu.replace('버거', '').strip():
            return save_menu(menu)
    return None

def add_to_cart(order_data, num):
    cart = {}
    cart["name"] = order_data["name"]
    cart["num"] = num
    cart["price"] = order_data["price"]
    cart["set_menu"] = order_data["set_menu"]
    if order_data["set_menu"]:
        cart["set_price"] = order_data["price"]  #나중에 set_price로 변경예정
    shopping_cart.append(cart)

intent = intent_chain.invoke(question)
parsed_intent = parse_response(intent)
category = parsed_intent.get('분류')
intent_keyword = parsed_intent.get('키워드')  # 키워드가 없으면 None

In [ ]:
question = "세트로 불고기 버거에 후렌치후라이 하나랑 콜라주고, 상하이 치킨 스낵랩도 하나 추가해줘"
intent = intent_chain.invoke(question)
parsed_intent = parse_response(intent)
category = parsed_intent.get('분류')
intent_keyword = parsed_intent.get('키워드')  # 키워드가 없으면 None
print(intent)
print(parsed_intent)
print(category)
print(intent_keyword)

{"분류": "주문", "키워드": "불고기 버거 세트, 후렌치후라이, 콜라, 상하이 치킨 스낵랩"}
{'분류': '주문', '키워드': '불고기 버거 세트, 후렌치후라이, 콜라, 상하이 치킨 스낵랩'}
주문
불고기 버거 세트, 후렌치후라이, 콜라, 상하이 치킨 스낵랩


In [ ]:
### 빠른주문 모듈
recommend = False
set_menu = False
menu_data = []
shopping_cart = []
unrecognized_keywords = []

if category == '주문':
    try:
        keyword_list = intent_keyword.split(',')
        print(f"keyword_list : {keyword_list}")
        for keyword in keyword_list:
            n_keyword = process_keyword(keyword)
            llm_keyword = process_keyword(name_chain.invoke(keyword)["answer"])
            print(f"keyword : {keyword}")
            print(f"n_keyword : {n_keyword}")
            print(f"llm_keyword : {llm_keyword}")
            menu_data_json = find_menu(n_keyword, menu_list) or find_menu(llm_keyword, menu_list)
            if menu_data_json is None:
                unrecognized_keywords.append(keyword)
                print(f"unrecognized_keywords : {unrecognized_keywords}")
                recommend = True
            else:
                menu_data_json = json.loads(menu_data_json) if isinstance(menu_data_json, str) else menu_data_json
                if "세트" in keyword:
                    menu_data_json['set_menu'] = True
                else:
                    menu_data_json['set_menu'] = False
                menu_data.append(menu_data_json)
                print(f"menu_data_json : {menu_data_json}")
                print('--------------------------------')
    except KeyError as e:
        print(f"키워드 없음: {e}")
        recommend = True
    except TypeError as e:
        print(f"TypeError 발생: {e}")
        recommend = True
    except Exception as e:  # 기타 에러 처리
        print(f"에러 발생: {e}")
        recommend = True
else:
    print("주문 의도가 아닙니다.")
    print(recommend)
    
if menu_data:
    for order_data in menu_data:
        num = input(f"{order_data['name']} 메뉴가 맞으신가요? 수량은 몇 개 드릴까요?")                
        add_to_cart(order_data, num)
    print(f"shopping_cart : {shopping_cart}")
    print('--------------------------------')
if unrecognized_keywords:
    print(f"선택하신 메뉴 중 확인할 수 없는 항목이 있습니다: {unrecognized_keywords}. 추천 챗봇을 연결해 드리겠습니다.")
    print('--------------------------------')

keyword_list : ['불고기 버거 세트', ' 후렌치후라이', ' 콜라', ' 상하이 치킨 스낵랩']
keyword : 불고기 버거 세트
n_keyword : 불고기
llm_keyword : 불고기
menu_data_json : {'name': '불고기 버거', 'category': '버거', 'description': '한국인의 입맛에 딱 맞는 불고기 소스에잘 재운 패티와\r\n고소한 마요네즈,신선한 양상추의 맛있는 조합.', 'price': '4200', 'nutrition': {'중량(g)': '167g', '중량(ml)': '-', '열량': '409kcal', '당': '14g', '단백질': '14g', '포화지방': '6g', '나트륨': '670mg', '카페인': '-'}, 'origin': '돼지고기:국내산', 'set_menu': True}
--------------------------------
keyword :  후렌치후라이
n_keyword : 후렌치후라이
llm_keyword : 후렌치 후라이
menu_data_json : {'name': '후렌치 후라이', 'category': '사이드', 'description': '통으로 썰어낸 감자를 맥도날드만의 노하우로 튀겨낸\r\n남다른 맛과 바삭함!맥도날드의 역사가 담긴 월드 클래스 후렌치 후라이', 'price': '3000', 'nutrition': {'중량(g)': '-', '중량(ml)': '-', '열량': '-', '당': '-', '단백질': '-', '포화지방': '-', '나트륨': '-', '카페인': '-'}, 'origin': '', 'set_menu': False}
--------------------------------
keyword :  콜라
n_keyword : 콜라
llm_keyword : 코카-콜라  
코카-콜라 제로
unrecognized_keywords : [' 콜라']
keyword :  상하이 치킨 스낵랩
n_keyword : 상하이 치

#### 메뉴명 추출

In [8]:
#메뉴명 추출

file_path = '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/menu_1014.json'

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 메뉴명을 저장할 리스트
menu_list = []
for item in data:
    menu_name = item['page_content']['name']
    menu_list.append(menu_name)

print(menu_list)

['맥크리스피 스리라차 마요', '베토디 스리라차 마요', '맥스파이시 상하이 버거', '토마토 치즈 비프 버거', '더블 쿼터파운더 치즈', '쿼터파운더 치즈', '빅맥', '맥크리스피 디럭스 버거', '맥크리스피 클래식 버거', '1955 버거', '맥치킨 모짜렐라', '맥치킨', '더블 불고기 버거', '불고기 버거', '슈슈 버거', '슈비 버거', '베이컨 토마토 디럭스', '치즈버거', '더블 치즈버거', '트리플 치즈버거', '햄버거', '소시지 스낵랩', '맥윙', '맥윙콤보', '코울슬로', '상하이 치킨 스낵랩', '골든 모짜렐라 치즈스틱', '후렌치 후라이', '맥너겟', '맥스파이시치킨 텐더', '해쉬 브라운', '애플 파이', '베리 스트로베리 맥플러리', '오레오 맥플러리', '딸기 오레오 맥플러리', '초코 오레오 맥플러리', '스트로베리콘', '아이스크림콘', '바닐라 선데이 아이스크림', '초코 선데이 아이스크림', '딸기 선데이 아이스크림', '오레오 아포가토', '바닐라 라떼', '아이스 바닐라 라떼', '카페라떼', '디카페인 카페라떼', '아이스 카페라떼', '디카페인 아이스 카페라떼', '아메리카노', '디카페인 아메리카노', '아이스 아메리카노', '디카페인 아이스 아메리카노', '카푸치노', '디카페인 카푸치노', '드립 커피', '아이스 드립 커피', '코카-콜라', '코카-콜라 제로', '스프라이트', '환타', '바닐라 쉐이크 Medium', '딸기 쉐이크 Medium', '초코 쉐이크 Medium', '오렌지 주스', '생수']


#### 이름추측 LLM model

In [9]:
def process_keyword(word):
    word = ''.join(word)
    word = word.replace("세트", "").strip()
    word = word.replace("버거", "").strip()
    return word

In [33]:
def save_menu(keyword):
    menu_data = None

    # 데이터를 순회하면서 키워드와 일치하는 아이템을 찾음
    for item in data:
        if item['page_content']['name'] == keyword:
            menu_data = {
                "name": item['page_content']['name'],
                "category": item['page_content']['category'],
                "description": item['page_content']['description'],
                "price": item['page_content']['price'],
                "nutrition": item['page_content']['nutrition'],
                "origin": item['page_content']['origin']
            }
            break

    # JSON 형식으로 출력 (또는 이후에 사용)
    if menu_data:
        menu_data_json = json.dumps(menu_data, ensure_ascii=False, indent=4)
        print(menu_data_json)
        return menu_data_json
    else:
        print("해당 키워드와 일치하는 메뉴를 찾을 수 없습니다.")

def find_menu(keyword, menu_list):
    if keyword in menu_list:
        return save_menu(keyword)
    return None

In [ ]:
# 커스텀 출력 파서 정의
class CustomOutputParser(BaseOutputParser):
    def parse(self, text: str) -> dict:
        # 항상 'answer' 키로 '정답'을 반환
        return {"answer": text.strip()}

# llm = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18", temperature=0.3)
llm = ChatAnthropic(model_name="claude-3-5-sonnet-20240620")
output_parser = CustomOutputParser()

# 프롬프트 템플릿 정의
name_chain = PromptTemplate.from_template("""
사용자의 질문과 메뉴 리스트를 비교하여 사용자의 질문과 비슷한 메뉴가 있는지 확인하세요.
비슷한 이름의 메뉴가 있다면 비슷한 메뉴를 모두 출력하세요
비슷한 메뉴가 없다면 "없음"이라고 출력하세요.
출력 시 메뉴 이름만 출력하세요

<메뉴 리스트>
['맥크리스피 스리라차 마요', '베토디 스리라차 마요', '맥스파이시 상하이 버거', '토마토 치즈 비프 버거', '더블 쿼터파운더 치즈', '쿼터파운더 치즈', '빅맥', '맥크리스피 디럭스 버거', '맥크리스피 클래식 버거', '1955 버거', '맥치킨 모짜렐라', '맥치킨', '더블 불고기 버거', '불고기 버거', '슈슈 버거', '슈비 버거', '베이컨 토마토 디럭스', '치즈버거', '더블 치즈버거', '트리플 치즈버거', '햄버거', '소시지 스낵랩', '맥윙', '맥윙콤보', '코울슬로', '상하이 치킨 스낵랩', '골든 모짜렐라 치즈스틱', '후렌치 후라이', '맥너겟', '맥스파이시치킨 텐더', '해쉬 브라운', '애플 파이', '베리 스트로베리 맥플러리', '오레오 맥플러리', '딸기 오레오 맥플러리', '초코 오레오 맥플러리', '스트로베리콘', '아이스크림콘', '바닐라 선데이 아이스크림', '초코 선데이 아이스크림', '딸기 선데이 아이스크림', '오레오 아포가토', '바닐라 라떼', '아이스 바닐라 라떼', '카페라떼', '디카페인 카페라떼', '아이스 카페라떼', '디카페인 아이스 카페라떼', '아메리카노', '디카페인 아메리카노', '아이스 아메리카노', '디카페인 아이스 아메리카노', '카푸치노', '디카페인 카푸치노', '드립 커피', '아이스 드립 커피', '코카-콜라', '코카-콜라 제로', '스프라이트', '환타', '바닐라 쉐이크 Medium', '딸기 쉐이크 Medium', '초코 쉐이크 Medium', '오렌지 주스', '생수']
</메뉴 리스트>

<질문>
{question}
</질문>

""") | llm | output_parser



In [ ]:
question = "빅맥 주문할게" 
intent = intent_chain.invoke(question)
print(parse_response(intent))

In [ ]:
intent = intent_chain.invoke("크리스탈 버거 하나")
print(parse_response(intent))
# 주문

{'분류': '주문', '키워드': '크리스탈 버거'}


In [ ]:
intent = intent_chain.invoke("베토디 세트 하나")
print(parse_response(intent))
# 주문

{'분류': '주문', '키워드': '베토디 세트'}


In [113]:
intent = intent_chain.invoke("불고기 버거 세트에 후렌치후라이 하나랑 콜라주고, 상하이 치킨 스낵랩도 하나 추가해줘")
print(parse_response(intent))
# 주문

{'분류': '주문', '키워드': '불고기 버거 세트, 후렌치후라이, 콜라, 상하이 치킨 스낵랩'}


In [ ]:
parsed_intent = parse_response(intent)
category = parsed_intent.get('분류')
keyword = parsed_intent.get('키워드')  # 키워드가 없으면 None

In [ ]:
print(category)
print(keyword)

주문
불고기 버거 세트, 후렌치후라이, 콜라, 상하이 치킨 스낵랩


In [ ]:
keywords = keyword.split(',')
print(type(keywords))
print(keywords)

<class 'list'>
['불고기 버거 세트', ' 후렌치후라이', ' 콜라', ' 상하이 치킨 스낵랩']


In [ ]:
keyword = "맥크리스탈 마요"
name_chain.invoke(keyword)

{'answer': '맥크리스피 스리라차 마요'}

In [ ]:
keyword = "맥크리스탈 디럭스"
name_chain.invoke(keyword)

{'answer': '맥크리스피 디럭스 버거'}

In [ ]:
keyword = "맥크리스탈 클래식"
name_chain.invoke(keyword)

{'answer': '맥크리스피 클래식 버거'}

In [ ]:
keyword = "맥크리스탈"
name_chain.invoke(keyword)
# 세가지나 있어서 아마 모르는것 같음

{'answer': '맥크리스피 스리라차 마요\n맥크리스피 디럭스 버거\n맥크리스피 클래식 버거'}

In [ ]:
keyword = "스파이시"
name_chain.invoke(keyword)


{'answer': '맥스파이시 상하이 버거\n맥스파이시치킨 텐더'}

In [ ]:
keyword = "베토디"
name_chain.invoke(keyword)
# 베이컨 토마토 디럭스도 있음.. ㅜㅜ 

{'answer': '베토디 스리라차 마요'}

#### 빠른주문 모듈

keyword가 없거나, menu list에 없으면 추천시스템으로 넘어가기

In [51]:
recommend = False
set_menu = False

if category == '주문':
    # 이름처리
    print(f"keyword : {keyword}")
    n_keyword = process_keyword(keyword)
    print(f"n_keyword : {n_keyword}")
    llm_keyword = process_keyword(name_chain.invoke(keyword)["answer"])
    print(f"llm_keyword : {llm_keyword}")

    # 키워드가 None일 경우 -> 추천모듈로 넘어가기
    if keyword is None or llm_keyword == "없음":
        recommend = True
        print("키워드 없음 또는 없음")
    else:
        try:
            menu_data_json = None
            # 세트메뉴인지 단품인지에 따라 처리
            if "세트" in keyword:
                set_menu = True
                menu_data_json = find_menu(n_keyword, menu_list) or find_menu(llm_keyword, menu_list)
                if menu_data_json is None:
                    recommend = True
                    print("세트 없음")
            else:  # 단품 메뉴인 경우
                menu_data_json = find_menu(n_keyword, menu_list) or find_menu(llm_keyword, menu_list)
                if menu_data_json is None:
                    recommend = True
                    print("메뉴 없음")
                else:
                    pass
        except KeyError as e:
            print(f"KeyError 발생: {e}")
            recommend = True
        except TypeError as e:
            print(f"TypeError 발생: {e}")
            recommend = True
        except Exception as e:  # 기타 에러 처리
            print(f"에러 발생: {e}")
            recommend = True
            
print(f"recommend : {recommend}")
print(f"set_menu : {set_menu}")


keyword : 베토디 세트
n_keyword : 베토디
llm_keyword : 베토디 스리라차 마요
{
    "name": "베토디 스리라차 마요",
    "category": "버거",
    "description": "베이컨 토마토 디럭스에스리라차 마요 소스를 더해 색다른 매콤함!",
    "price": "7600",
    "nutrition": {
        "중량(g)": "251g",
        "중량(ml)": "-",
        "열량": "621kcal",
        "당": "13g",
        "단백질": "29g",
        "포화지방": "12g",
        "나트륨": "985mg",
        "카페인": "-"
    },
    "origin": "쇠고기:호주산돼지고기(베이컨):외국산(아일랜드,스페인,캐나다)"
}
recommend : False
set_menu : True


In [57]:
menu_data_json = json.loads(menu_data_json) if isinstance(menu_data_json, str) else menu_data_json
print(f"type : {type(menu_data_json)}")
print(f"menu_data_json : {menu_data_json}")


type : <class 'dict'>
menu_data_json : {'name': '베토디 스리라차 마요', 'category': '버거', 'description': '베이컨 토마토 디럭스에스리라차 마요 소스를 더해 색다른 매콤함!', 'price': '7600', 'nutrition': {'중량(g)': '251g', '중량(ml)': '-', '열량': '621kcal', '당': '13g', '단백질': '29g', '포화지방': '12g', '나트륨': '985mg', '카페인': '-'}, 'origin': '쇠고기:호주산돼지고기(베이컨):외국산(아일랜드,스페인,캐나다)'}


In [58]:
menu_data_json["name"]

'베토디 스리라차 마요'

### 장바구니 넣기(장바구니 모듈 python 만들기)

단품, 세트 각각 딕셔너리에 묶어서 [[{단품1}], [{세트버거1},{세트사이드1},{세트음료1}],[{단품2}]]로 구성


In [59]:
# 주문데이터 장바구니에 넘겨주기
shopping_cart = []

def add_to_cart(order_data,num):
    cart = {}
    cart["name"] = order_data["name"]
    cart["num"] = num
    cart["price"] = order_data["price"]
    shopping_cart.append(cart)

In [60]:
add_to_cart(menu_data_json, 1)

In [61]:
print(shopping_cart)

[{'name': '베토디 스리라차 마요', 'num': 1, 'price': '7600'}]


## 3. 추천 시스템, memory & langgraph test

In [227]:
file_dir = "/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/menu_1014 copy.json"

with open(file_dir, 'r', encoding='utf-8') as file:
    content = file.read()
    # JSON 객체들을 리스트로 변환
    json_objects = json.loads(content)

# 파싱된 JSON 객체들을 문서로 변환
docs = [
    Document(
        page_content=json.dumps(obj['page_content'], ensure_ascii=False),
    )
    for obj in json_objects
]

file_name = file_dir.split("/")[-1]
cache_dir = LocalFileStore(f"./.cache/embeddings/{file_name}")

# CharacterTextSplitter를 사용하여 문서 분할
text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=100, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large"
)

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=cache_dir,
    namespace="solar-embedding-1-large",
)
vectorstore = FAISS.from_documents(
    split_docs,
    cached_embedder,
)

faiss = vectorstore.as_retriever(search_kwargs={"k": 4})

bm25 = BM25Retriever.from_documents(split_docs)
bm25.k = 4

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25, faiss],
    weights=[0.3, 0.7],
    search_type="mmr",
)

In [228]:
print(split_docs[0])

page_content='{"name": "맥크리스피 스리라차 마요", "category": "버거", "description": "빠삭한 100% 통닭다리살 케이준 패티에스리라차 마요 소스를 더해 매콤 고소한 맛!", "price": "8600", "set_burger_price": "4900", "new_menu": "신제품", "nutrition": {"중량(g)": "289g", "중량(ml)": "-", "열량": "663kcal", "당": "15g", "단백질": "26g", "포화지방": "7g", "나트륨": "1508mg", "카페인": "-"}, "origin": "닭고기:태국산(경상지역), 브라질산(그 외 지역)"}'


In [229]:
# 메시지를 저장하는 리스트

def save_message(message, role):
    messages.append({"message": message, "role": role})
    
def get_memory():
    return ConversationBufferMemory(
        return_messages=True,
        memory_key="chat_history"
    )

In [230]:
llm = ChatAnthropic(model_name="claude-3-5-sonnet-20240620")
memory = get_memory()

In [232]:
# recommend_template = ChatPromptTemplate.from_messages([
#     ("system",
#     """
#     당신은 맥도날드 가게의 점원입니다. 
#     나이가 많은 노인 고객의 주문을 도와주세요.
#     나이와 관련된 어떤한 호칭도 하지 말고 '고객님'으로 부르세요.
#     어린아이도 이해하기 쉬운 단어로 설명해주세요.
#     상냥하고, 친절하고, 존댓말로 사용자의 질문에 답변을 해주세요. 
#     간결하게 답변해주세요.
    
#     반드시 제공된 정보만을 사용하여 질문에 대답하세요.
#     질문과 가장 관련성이 높은 정보를 찾아서 답변하세요.
#     메뉴추천은 최대 4개까지만 추천해주세요.
#     충분한 정보를 바탕으로 정확히 답변할 수 있는 경우에만 답변하세요.
#     항상 chat history를 고려하여 이전 대화내용과 관련되게 대답하세요

#     답변이 확실하지 않을 경우, "죄송합니다. 해당 질문에 대한 답을 찾을 수 없습니다."라고 답하세요.
    
#     주문 조건
#     - 원하는 메뉴가 없이 추천을 받고싶으면 신메뉴를 추천해주세요.
#     - 주문메뉴는 세트와 단품으로 구분한다.
#     - 세트메뉴는 버거, 사이드, 음료가 반드시 포함되어야 한다.
#     - 세트메뉴는 기본 미디엄 사이즈로 구성되며, 라지 사이즈로 변경 시 800원의 차액이 부과된다.
#     - 세트메뉴의 기본 구성 사이드는 후렌치 후라이 미디엄 1개, 음료는 코카콜라 미디엄 1개이다.
#     - 세트메뉴의 사이드는 코울슬로로 변경 가능하며 추가금액은 없다.
#     - 세트메뉴의 음료는 '음료'카테고리 내에서 변경 가능하며, 변경할 경우 해당 메뉴끼리의 차액이 추가되는 점을 안내해 드린다.
#     - 세트메뉴의 가격을 물어보면 'set_burger_price'의 값에 5600원을 더한 총 금액을 안내해 드린다.
#     - 단품 후렌치후라이는 스몰, 미디엄, 라지 중 하나를 선택할 수 있으며, 스몰 사이즈는 2300원, 미디움 사이즈는 3000원, 라지 사이즈는 4000원이다.
#     - 단품 음료는 미디엄, 라지 중 하나를 선택할 수 있으며, 미디엄 사이즈는 2600원, 라지 사이즈는 3100원이다.
#     - 추가 주문이 필요한지 묻고, 추가 주문이 없으면 최종 주문결과를 출력한다.
    
#     주문결과 출력 :
#     - 주문 결과는 세트메뉴와 단품메뉴를 각각 리스트로 묶어서 출력한다.
#     - 주문 결과는 json 형식으로 출력한다.

#     주문 결과 예시: 
#     [
#         [
#             {{'name': '불고기 버거', 'num': '1', 'price': '4200', 'set_menu': True, 'set_burger_price': '4000'}}, 
#             {{'name': '후렌치 후라이', 'num': '1', 'price': '3000'}},
#             {{'name': '코카콜라', 'num': '1', 'price': '2600'}}
#         ],
#         [
#             {{'name': '상하이 치킨 스낵랩', 'num': '1', 'price': '4000'}}
#         ]
#     ]
#     Context: {context}
#     Chat History: {chat_history}
#     """),
#     ("human", "{question}"),
# ])

In [235]:
recommend_template = ChatPromptTemplate.from_messages([
    ("system",
    """
    당신은 맥도날드의 친절한 점원입니다.
    고객님의 주문을 도와드리세요. 나이와 상관없이 '고객님'이라고만 부르고, 모든 설명은 어린아이도 이해할 수 있게 간단하게 해주세요. 친절하고 상냥하게 존댓말로 응대합니다.
    
    대답할 때는 다음 사항을 기억하세요:
    1. 제공된 정보만으로 답변합니다.
    2. 질문과 가장 관련성이 높은 정보를 찾아서 답변하세요.
    3. 메뉴 추천은 4개 이하로 제한하고, 신메뉴를 먼저 추천하세요.
    4. 항상 대화 기록을 고려해 대답하세요.
    5. 확실하지 않은 경우 "죄송합니다. 해당 질문에 대한 답을 찾을 수 없습니다."라고 답하세요.

    **주문 조건:**
    - 메뉴는 세트와 단품으로 구분됩니다.
    - 세트메뉴는 버거, 사이드, 음료가 포함되며, 미디엄 사이즈가 기본입니다.
    - 사이즈 업그레이드는 800원 추가 요금이 부과됩니다.
    - 사이드는 후렌치 후라이 미디엄이 기본이며, 코울슬로로 무료 변경 가능합니다.
    - 음료는 코카콜라 미디엄이 기본이며, 음료 변경 시 차액이 발생할 수 있습니다.
    - 세트메뉴 가격은 'set_burger_price'에 5600원을 더한 금액입니다.
    - 단품 후렌치 후라이는 스몰(2300원), 미디엄(3000원), 라지(4000원) 중 선택할 수 있습니다.
    - 단품 음료는 미디엄(2600원) 또는 라지(3100원)입니다.

    **주문 절차:**
    - 먼저 고객의 주문을 받고 메뉴가 완성되면 추가 주문 또는 결제여부를 물어보세요.
    - 메뉴의 주문이 완성되지 않은 경우 추가 주문 또는 결제여부를 묻지 마세요.
    - 주문이 모두 완료되면 최종 주문 결과를 출력하세요.

    **주문 결과 출력:**
    - 세트와 단품 메뉴를 각각 리스트로 구분해 JSON 형식으로 출력합니다.
    
    **주문 결과 예시:**
    [
        [
            {{'name': '불고기 버거', 'num': '1', 'price': '4200', 'set_menu': True, 'set_burger_price': '4000'}}, 
            {{'name': '후렌치 후라이', 'num': '1', 'price': '3000'}},
            {{'name': '코카콜라', 'num': '1', 'price': '2600'}}
        ],
        [
            {{'name': '상하이 치킨 스낵랩', 'num': '1', 'price': '4000'}}
        ]
    ]

    **대화 예시:**
    - 고객: "추천 메뉴가 있나요?"
    - AI: "신메뉴로 '1955 버거 세트', '맥스파이시 상하이 버거 세트', '베이컨 토마토 디럭스 세트', '상하이 치킨 스낵랩 세트'를 추천드립니다."

    - 고객: "1955 버거 세트 가격이 얼마인가요?"
    - AI: "1955 버거 세트는 9600원입니다. 후렌치 후라이 미디엄과 코카콜라 미디엄이 포함됩니다."

    **Context:** {context}
    **Chat History:** {chat_history}
    """),
    ("human", "{question}"),
]) 


In [233]:
recommend_chain = {
    "context": ensemble_retriever,
    "chat_history": RunnableLambda(lambda _: memory.load_memory_variables({})["chat_history"]),
    "question": RunnablePassthrough()
} | recommend_template | llm

In [237]:
messages = []
memory.clear()

while True:
    # 사용자로부터 입력 받기
    user_message = input("질문을 입력해주세요: ").strip()

    # 입력이 비어있는 경우 반복문을 계속 진행
    if not user_message:
        continue
    
    save_message(user_message, "user")
    
    response = recommend_chain.invoke(user_message)
    ai_response = response.content if hasattr(response, 'content') else response
    save_message(ai_response, "assistant")
    print(f"AI : {ai_response}")
    
    memory.save_context({"input": user_message}, {"output": ai_response})

    # 주문 결과가 json 형식인지 확인하여 while문 종료
    try:
        # 문자열을 JSON으로 변환 시도
        parsed_response = json.loads(ai_response)
        
        # JSON 변환이 성공하면 프로그램 종료
        print("주문이 완료되었습니다. 프로그램을 종료합니다.")
        break
    except json.JSONDecodeError:
        # JSON 변환 실패 시, 반복 계속
        pass


AI : 안녕하세요, 고객님. 매콤한 메뉴를 찾으시는군요. 저희 맥도날드에서 매콤한 맛을 즐기실 수 있는 신메뉴를 추천해 드리겠습니다.

1. 맥크리스피 스리라차 마요 버거: 빠삭한 100% 통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤하고 고소한 맛을 즐기실 수 있습니다.

2. 베토디 스리라차 마요 버거: 베이컨 토마토 디럭스에 스리라차 마요 소스를 더해 색다른 매콤함을 느끼실 수 있습니다.

3. 맥스파이시 치킨텐더: 부드러운 100% 닭안심살을 스파이시 시즈닝으로 매콤 바삭하게 튀겨낸 치킨 텐더입니다.

4. 상하이 치킨 스낵랩: 매콤한 치킨에 달콤한 화이트 마요 소스로 매콤달콤한 맛을 즐기실 수 있습니다.

이 중에서 어떤 메뉴가 마음에 드시나요? 더 자세한 설명이 필요하시면 말씀해 주세요.


In [1]:
memory.load_memory_variables({})["chat_history"]

NameError: name 'memory' is not defined

In [224]:
print(messages)

[{'message': '', 'role': 'user'}]
